In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                          11/2020     Adrien Poindron    #
###########################################################

# Imports

In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [2]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt
from pathlib import Path

In [3]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [4]:
from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from scipy.signal import savgol_filter

In [5]:
def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    C1 = q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a ) ) ) ) )
    C2 = q**2/( (beta_guess-2)**2 - a - q**2/( (beta_guess-4)**2 - a - q**2/( (beta_guess-6)**2 - a - q**2/( (beta_guess-8)**2 - a - q**2/( (beta_guess-10)**2 - a  ) ) ) ) )
    return a + C1 + C2

In [6]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
# %run ../../Functions/1.2/data_fct_Adrien_point_by_point
%run ../GiantMol/Functions/1.2/data_fct_Adrien_point_by_point

matplotlib.rcParams.update({'font.size': 25})

In [7]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.00e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23  # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [8]:
renorm = 0.5*m_GM/C_e

# Sélection fichiers et chargement des données

In [9]:
def load_file_GUI(dir_string):

    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename(initialdir = dir_string,
                                       multiple=True)
    return file_path

In [10]:
file_path = load_file_GUI('/home/adrian/Documents/Programmes/Python/GiantMol/SimAnalysisPointbyPoint')
print(file_path)

('/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC01_RF08_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC01_RF09_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC01_RF10_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC01_RF11_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC03_RF08_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC03_RF09_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC03_RF10_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC03_RF11_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC05_RF08_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC05_RF09_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC05_RF10_50eV.npz', '/home/adrian/RemoteFS/Rivendel/Simulations/20200903/All_stat_DC05_RF11_50eV.npz', '/h

In [11]:
# load check
time         = []
Temperature  = []
Temp_var     = []
Fluorescence = []
Fluo_var     = []
SNR          = []
Ray          = []
Len          = []
DeltaEc      = []
condition    = []

for outfile in file_path:
    with load(outfile) as data:
        time.append(data['time'])
        Temperature.append(data['Temperature'])
        Temp_var.append(data['Temp_var'])
        Fluorescence.append(data['Fluo'])
        Fluo_var.append(data['Fluo_var'])
        SNR.append(data['SNR'])
        Ray.append(data['Ray'])
        Len.append(data['Len'])
        DeltaEc.append(data['DeltaEc'])
        condition.append(data['condition'])

DeltaEc = multiply(DeltaEc,renorm)
# Temperature,Temp_var,
# Fluo, Fluo_var, SNR,
# Ray, Len,
# DeltaEc,
# condition

In [12]:
E0_GMol = 50
kappa = 0.23
print(condition)
my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]

[array('DC01_RF08', dtype='<U9'), array('DC01_RF09', dtype='<U9'), array('DC01_RF10', dtype='<U9'), array('DC01_RF11', dtype='<U9'), array('DC03_RF08', dtype='<U9'), array('DC03_RF09', dtype='<U9'), array('DC03_RF10', dtype='<U9'), array('DC03_RF11', dtype='<U9'), array('DC05_RF08', dtype='<U9'), array('DC05_RF09', dtype='<U9'), array('DC05_RF10', dtype='<U9'), array('DC05_RF11', dtype='<U9'), array('DC07_RF08', dtype='<U9'), array('DC07_RF09', dtype='<U9'), array('DC07_RF10', dtype='<U9'), array('DC07_RF11', dtype='<U9'), array('DC09_RF08', dtype='<U9'), array('DC09_RF09', dtype='<U9'), array('DC09_RF10', dtype='<U9'), array('DC09_RF11', dtype='<U9'), array('DC11_RF08', dtype='<U9'), array('DC11_RF09', dtype='<U9'), array('DC11_RF10', dtype='<U9'), array('DC11_RF11', dtype='<U9'), array('DC18_RF08', dtype='<U9'), array('DC18_RF09', dtype='<U9'), array('DC18_RF10', dtype='<U9'), array('DC18_RF11', dtype='<U9')]


In [13]:
detect_rate = []

for k,l in enumerate(SNR):
    alors = 0
    for m,n in enumerate(l):
        if n>=1:
            alors += 1
    detect_rate.append(alors)

# Traitement des données

In [14]:
shape(time),shape(Temperature),shape(Fluorescence)

((28, 100, 20850), (28, 100, 20850, 3), (28, 100, 19860))

In [15]:
cond_num = len(condition)

plot_point = 10
plot_try   = 28

tt    = time[plot_point][plot_try]
T_all = Temperature[plot_point][plot_try]
PM    = Fluorescence[plot_point][plot_try]

print(shape(tt),shape(T_all),shape(PM))

n_dt = 100
dt  = 2*pi / (n_dt*Omega)
i_free__fly_Lan = 1000*n_dt
i_laser_fly = 6000*n_dt
i_laser_fly_quench = 8000*n_dt
print(dt)
diff_time = len(tt)-len(PM)
print(diff_time)

(20850,) (20850, 3) (19860,)
5e-09
990


In [16]:
# Plot Delta E en fct L
# Avec discri detec et non detec

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Plot temperature Langevin + md laser  ALL SimuType  ALL TEMP MEAN
matplotlib.rcParams.update({'font.size': 25})

#fig_name = file_name[-9:]
figname = 'T_and_PM_plot_example'
fig1 = plt.figure(num=figname,clear='True')
fig1.set_size_inches(11.69,8.27)
# set height ratios for sublots
gs = gridspec.GridSpec(2, 1, height_ratios=[7, 3]) 
# the fisrt subplot
ax1 = plt.subplot(gs[0])

ax1.semilogy(tt*1e3,savgol_filter(mean(T_all,axis=1),25,1), label='Tx',color='red',lw=2.5)

time_switch = i_free__fly_Lan*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
ax1.vlines(time_switch*1e3,1e-5,1e4,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
ax1.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax1.vlines((ta[2])*1e3,1e-4,1e4,color='black')
# ax1.hlines(0.5,7.5,8.5, linestyle=':')
# ax1.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,1e4,color='purple')
ax1.grid()
ylabel('T [K]')

ax1.annotate('1', xy=(0.22,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('2', xy=(2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('3', xy=((3.5+7.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
# ax1.annotate('4', xy=((7.5+7.57)/2,2.5e-4), xycoords='data',
# size=15, ha='center', va='top', color='xkcd:black',
# bbox=dict(fc='white',edgecolor='black'))
ax1.annotate('5', xy=((7.5+10.5)/2,1.3e-4), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

# annotate('Laser ON', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul')

# ~ xlabel('time[ms]')
# ~ ylabel('T[K]')
plt.setp(ax1.get_xticklabels(),visible=False)

# ax2 = subplot(212,sharex=ax1)
ax2 = plt.subplot(gs[1], sharex = ax1)

# ax2.plot(tt_md*1e3,PM, label='Tx')

tt_1ms_step = tt[diff_time::2000]
PM_1ms = PM[::2000]
tt_1us_step = tt[diff_time::2]
PM_1us = PM[::2]

ax2.scatter(tt_1ms_step*1e3,PM_1ms, label='cts/ms',marker='P',s=200,lw=1.5,color='black',zorder=100)
ax2.plot(tt_1us_step[:-500]*1e3,PM_1us[:-500], label='cts/µs',color='blue')
ax2.legend()
time_switch = i_free__fly_Lan*dt
ax2.vlines(time_switch*1e3,1e-4,150,color='black')
time_switch = (i_free__fly_Lan+i_laser_fly)*dt
# ax2.vlines(time_switch*1e3,1e-4,150,color='black',linestyle='--')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
# ax2.vlines(time_switch*1e3,1e-4,70,color='orange')
time_switch = (i_free__fly_Lan+i_laser_fly*2+i_laser_fly_quench)*dt
# ax2.vlines((time_switch+ta[2]-ta[1])*1e3,1e-4,90,color='purple')
ax2.grid()
# ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=20))
# annotate('laser off après injection', xy=(0.5,350), xycoords='data',
# size=24, ha='left', va='top', color='xkcd:azul',
# bbox=dict(boxstyle='round', fc='white',edgecolor='xkcd:azul'))

xlabel('time [ms]')
ylabel('PM signal')

left, bottom, width, height = [0.345, 0.565, 0.35, 0.38] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

axins.semilogy(tt*1e3,mean(T_all,axis=1), label='Tx',color='red',lw=2.5,marker='x')
# axins.hlines(0.5,7.4,7.66, linestyle=':')
time_switch = (i_free__fly_Lan+i_laser_fly+i_laser_fly_quench)*dt
axins.vlines(time_switch*1e3,1e-4,1e4,color='black',linestyle='-')
time_switch = (i_free__fly_Lan+i_laser_fly  +i_laser_fly_quench)*dt
# axins.vlines(ta[2]*1e3,1e-4,1e4,color='black')

# axins.add_patch(
#      patches.Rectangle(
#         (t_in*1000, 3e-4),
#         (t_out - t_in)*1000,
#         0.1,
#         edgecolor ='k',
#         facecolor =  cc('grey',0.5),
#         fill=True
#      ) )

x1, x2, y1, y2 = 7.44, 7.63, 0.00012, 50 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# axins.yaxis.tick_right()

locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
axins.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
axins.yaxis.set_minor_locator(locmin)
axins.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
plt.setp(axins.get_xticklabels(), backgroundcolor="w")
plt.setp(axins.get_yticklabels(), backgroundcolor="w")

axins.grid(b=True, which='major', linestyle='-')
axins.grid(b=True, which='minor', linestyle='--')

# axins.tick_params(axis="y",direction="in", pad=-53)
mark_inset(ax1, axins, loc1=1, loc2=3, fc="none", ec="0.25")

axins.annotate('3', xy=(7.48,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('4', xy=((7.5+7.53)/2,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))
axins.annotate('5', xy=(7.55,30), xycoords='data',
size=15, ha='center', va='top', color='xkcd:black',
bbox=dict(fc='white',edgecolor='black'))

ax1.set_xlim(-0.15, 10.75)
ax1.set_ylim(0.00005, 1000)
ax2.set_ylim(-0, 135)

plt.tight_layout()
subplots_adjust(hspace=0.0,bottom=0.11,left=0.12,right=0.98,top=0.97)
# savefig(fname = figname+'_'+re.sub('/','_',dirname_hc[31:-1])+'.png',dpi=300)
# savefig(figname+'_D'+str(Detuning)+'_50eV'+'.png',dpi=300)

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:142: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [77]:
# Plot Delta en fct omega_r

points_and_coord = {'DC01_RF08': ['01', '08'],
 'DC01_RF09': ['01', '09'],
 'DC01_RF10': ['01', '10'],
 'DC01_RF11': ['01', '11'],
 'DC03_RF08': ['03', '08'],
 'DC03_RF09': ['03', '09'],
 'DC03_RF10': ['03', '10'],
 'DC03_RF11': ['03', '11'],
 'DC05_RF08': ['05', '08'],
 'DC05_RF09': ['05', '09'],
 'DC05_RF10': ['05', '10'],
 'DC05_RF11': ['05', '11'],
 'DC07_RF08': ['07', '08'],
 'DC07_RF09': ['07', '09'],
 'DC07_RF10': ['07', '10'],
 'DC07_RF11': ['07', '11'],
 'DC09_RF08': ['09', '08'],
 'DC09_RF09': ['09', '09'],
 'DC09_RF10': ['09', '10'],
 'DC09_RF11': ['09', '11'],
 'DC11_RF08': ['11', '08'],
 'DC11_RF09': ['11', '09'],
 'DC11_RF10': ['11', '10'],
 'DC11_RF11': ['11', '11'],
 'DC18_RF08': ['18', '08'],
 'DC18_RF09': ['18', '09'],
 'DC18_RF10': ['18', '10'],
 'DC18_RF11': ['18', '11']}

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]

x = []
y = []
z = []

omegaz_2 = zeros(len(list(points_and_coord)))
omegax = zeros(len(list(points_and_coord)))
omegap_2 = zeros(len(list(points_and_coord)))

for g,h in enumerate(points_and_coord):
    my_udc = Udc[int(points_and_coord[h][0])]
    my_urf = Urf[int(points_and_coord[h][1])]
    x.append(my_udc)
    y.append(my_urf)
    z.append(detect_rate[g])

    
    beta_guess = 0.402
#     a = 8*C_e*kappa/(m_Ca*d0**2*Omega**2)*my_udc
    a = 0
    q = 4*C_e/(m_Ca*r0**2*Omega**2)*my_urf
    for k in range(15):
        beta_guess = beta_continue_alamano(a,q,beta_guess)
        beta_guess = sqrt(beta_guess)
    print(beta_guess)

    omegax[g] = 0.5*beta_guess*Omega
    omegaz_2[g] = 2*kappa*my_udc*C_e/(m_Ca*d0**2)
    omegap_2[g] = omegax[g]**2 - 0.5*omegaz_2[g]
    
xL = mean(Len,axis=1)*1000
yfx = omegax /2/pi

0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833
0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833
0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833
0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833
0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833
0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833
0.3953515610162703
0.4418091979592634
0.49118672052083756
0.544606486211833


In [92]:
xL

array([380.63441837, 388.87737695, 395.94693127, 402.34305919,
       273.30711815, 282.51192408, 289.41200279, 295.40532108,
       219.83442633, 227.99040842, 236.3928712 , 243.606327  ,
       188.17885618, 196.00437223, 202.69656474, 208.60346411,
       164.96736041, 173.38108147, 180.44780753, 185.93778415,
       132.61265899, 141.15457193, 148.65637591, 155.00307152,
       146.97393442, 155.66334381, 162.8463366 , 169.01810552])

In [94]:
# x.append(L)
# y.append(my_fx)
# z.append(detect_rate[g])
        
cluster = array([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])
my_marker = ['v','s','p','H']

x=np.array(xL)
y=np.array(yfx)*1e-3
z=np.array(detect_rate,dtype='int')

fname = '2D_GMol_var_fluo_freqx_L'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

for kk in range(4):
    im = ax.scatter(list(x[cluster==kk+1])+[0,0],list(y[cluster==kk+1])+[0,0],
        marker=my_marker[kk%4],s=600,edgecolor='black',zorder = 1000,
        c = list(z[cluster==kk+1])+[0,100], cmap='viridis') # discrete_cmap(len(num_runs), 'jet')
ax.set_xlabel(r'$\langle L \rangle /2$ [µm]')
ax.set_ylabel(r'$f_{x}$ [kHz]')
ax.grid(visible='both')

for ii,jj in enumerate(my_order):
    ax.text(x[ii],y[ii]+12,str(ii),zorder = 10000,
            color='xkcd:black',fontsize='12',weight="bold",
            ha='center',va='center')

imbar = ax.scatter(x,y,
        marker=my_marker[1],s=0,edgecolor='black',zorder = 1000,
        c = z, cmap='viridis')
cbar = plt.colorbar(imbar, ax=ax)
# cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
cbar.ax.set_ylabel(r'Detection rate [%]', rotation=90)

ax.set_xlim(125,410)
ax.set_ylim(380,565)
ax.annotate(r'$ \mathbf{q_x=0.6} $', (0.22,64.3),fontsize=15,weight="bold")
ax.annotate(r'$\mathbf{q_x=0.5}$', (0.22,53.6),fontsize=15,weight="bold")

# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(14,6)

fig.tight_layout()
subplots_adjust(hspace=0.10, bottom=0.163, top = 0.94,left=0.10,right=1.03)
savefig('2D_GMol_var_fluo_freqx_L_D1_0_50eV.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.eps',dpi=300,format='eps')

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 


In [26]:
# x.append(L)
# y.append(my_fx)
# z.append(detect_rate[g])
        
cluster = array([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])
my_marker = ['v','s','p','H']

x=np.array(sqrt(omegaz_2)/2/pi)*1e-3
y=np.array(yfx)*1e-3
z=np.array(detect_rate,dtype='int')

fname = '2D_GMol_var_fluo_freqx_freqz'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

for kk in range(4):
    im = ax.scatter(list(x[cluster==kk+1])+[0,0],list(y[cluster==kk+1])+[0,0],
        marker=my_marker[kk%4],s=600,edgecolor='black',zorder = 1000,
        c = list(z[cluster==kk+1])+[0,100], cmap='viridis') # discrete_cmap(len(num_runs), 'jet')
ax.set_xlabel(r'$f_{z}$ [kHz]')
ax.set_ylabel(r'$f_{x}$ [kHz]')
ax.grid(visible='both')

for ii,jj in enumerate(my_order):
    ax.text(x[ii],y[ii]+13.5,str(ii),zorder = 10000,
            color='xkcd:black',fontsize='12',weight="bold",
            ha='center',va='center')

imbar = ax.scatter(x,y,
        marker=my_marker[1],s=0,edgecolor='black',zorder = 1000,
        c = z, cmap='viridis')
cbar = plt.colorbar(imbar, ax=ax)
# cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
cbar.ax.set_ylabel(r'Detection rate [%]', rotation=90)

ax.set_xlim(70,230)
ax.set_ylim(380,565)
ax.annotate(r'$ \mathbf{q_x=0.6} $', (0.22,64.3),fontsize=15,weight="bold")
ax.annotate(r'$\mathbf{q_x=0.5}$', (0.22,53.6),fontsize=15,weight="bold")

# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(14,6)

fig.tight_layout()
subplots_adjust(hspace=0.10, bottom=0.163, top = 0.94,left=0.10,right=1.03)
savefig('2D_GMol_var_fluo_freqx_freqz_D1_0_50eV.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.eps',dpi=300,format='eps')

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 


In [95]:
z

array([-0.02194448, -0.02772925, -0.03513065, -0.04128209, -0.01897329,
       -0.04117643, -0.02938714, -0.02171292, -0.00945064, -0.01266775,
       -0.02001991, -0.03517142, -0.01710595, -0.0088239 , -0.01080696,
       -0.01436896, -0.00779019, -0.01770155, -0.00852097, -0.01028037,
       -0.00360305, -0.00419978, -0.00951352, -0.01628801, -0.00429723,
       -0.00788992, -0.01821858, -0.00824137])

In [96]:
cluster = array([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])
my_marker = ['v','s','p','H']

x=np.array(xL)
y=np.array(yfx)*1e-3
z=np.array(mean(DeltaEc,axis=1))*1e3

fname = '2D_GMol_var_ener_freqx_L'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

for kk in range(4):
    im = ax.scatter(list(x[cluster==kk+1])+[0,0],list(y[cluster==kk+1])+[0,0],
        marker=my_marker[kk%4],s=600,edgecolor='black',zorder = 1000,
        c = list(z[cluster==kk+1])+[0,min(z)], cmap='Purples_r') # discrete_cmap(len(num_runs), 'jet')
ax.set_xlabel(r'$\langle L \rangle /2$ [µm]')
ax.set_ylabel(r'$f_{x}$ [kHz]')
ax.grid(visible='both')

for ii,jj in enumerate(my_order):
    ax.text(x[ii],y[ii]+13.5,str(ii),zorder = 10000,
            color='xkcd:black',fontsize='12',weight="bold",
            ha='center',va='center')

imbar = ax.scatter(x,y,
        marker=my_marker[1],s=0,edgecolor='black',zorder = 1000,
        c = z, cmap='Purples_r')
cbar = plt.colorbar(imbar, ax=ax)
# cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
cbar.ax.set_ylabel(r'Mean $\Delta E$ [meV]', rotation=90)

ax.set_xlim(125,410)
ax.set_ylim(380,565)
ax.annotate(r'$ \mathbf{q_x=0.6} $', (0.22,64.3),fontsize=15,weight="bold")
ax.annotate(r'$\mathbf{q_x=0.5}$', (0.22,53.6),fontsize=15,weight="bold")

# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(14,6)

fig.tight_layout()
subplots_adjust(hspace=0.05, bottom=0.163, top = 0.94,left=0.10,right=1.02)
savefig('2D_GMol_var_ener_freqx_L_D1_0_50eV.png',dpi=300)

# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.eps',dpi=300,format='eps')

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 


In [58]:
list(z[cluster==kk+1])+[0,min(z)]

[-41.28209375061351,
 -21.712916779611177,
 -35.171422924965384,
 -14.368957433694924,
 -10.280368757697158,
 -16.28801309610591,
 -8.241368035772208,
 0,
 -41.28209375061351]

In [64]:
cluster = array([1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4])
my_marker = ['v','s','p','H']

x=np.array(sqrt(omegaz_2)/2/pi)*1e-3
y=np.array(yfx)*1e-3
z=np.array(mean(DeltaEc,axis=1))*1e3

fname = '2D_GMol_var_ener_freqx_freqz'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
# title(all_subdir[0][myslashpos[slashcond-1]:-15]+'__E0='+str(E0_GMol)+'eV',fontsize=25)

for kk in range(4):
    im = ax.scatter(list(x[cluster==kk+1])+[0,0],list(y[cluster==kk+1])+[0,0],
        marker=my_marker[kk%4],s=600,edgecolor='black',zorder = 1000,
        c = list(z[cluster==kk+1])+[0,min(z)], cmap='Purples_r') # discrete_cmap(len(num_runs), 'jet')
ax.set_xlabel(r'$f_{z}$ [kHz]')
ax.set_ylabel(r'$f_{x}$ [kHz]')
ax.grid(visible='both')

for ii,jj in enumerate(my_order):
    ax.text(x[ii],y[ii]+13.5,str(ii),zorder = 10000,
            color='xkcd:black',fontsize='12',weight="bold",
            ha='center',va='center')

imbar = ax.scatter(x,y,
        marker=my_marker[1],s=0,edgecolor='black',zorder = 1000,
        c = z, cmap='Purples_r')
cbar = plt.colorbar(imbar, ax=ax)
# cbar.set_clim(min(z),0)
# cbar.clim((min(z),0))
# cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(detect_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
cbar.ax.set_ylabel(r'Mean $\Delta E$ [meV]', rotation=90)

ax.set_xlim(70,230)
ax.set_ylim(380,565)
ax.annotate(r'$ \mathbf{q_x=0.6} $', (0.22,64.3),fontsize=15,weight="bold")
ax.annotate(r'$\mathbf{q_x=0.5}$', (0.22,53.6),fontsize=15,weight="bold")

# ax.grid()
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
fig.set_size_inches(14,6)

fig.tight_layout()
subplots_adjust(hspace=0.05, bottom=0.163, top = 0.94,left=0.10,right=1.02)
# savefig('2D_GMol_var_ener_freqx_freqz_D1_0_50eV.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.png',dpi=300)
# savefig(fname+'_D1.0'+'_'+str(E0_GMol)+'eV'+'.eps',dpi=300,format='eps')

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 


# Histogram

In [33]:
shape(SNR)

(28, 100)

In [53]:
# Plot DE_hist

figname = 'DE_hist'
fig, ax = plt.subplots(1, 1,num=figname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(14,6)

my_condition = 22

y_all = DeltaEc[my_condition]*1000
y_detec    = y_all[SNR[my_condition]>=1]
y_nondetec = y_all[SNR[my_condition]<1]
my_bins = arange(-50,0,1)
ax.hist(y_detec,bins=my_bins ,color='xkcd:green', alpha=0.6, hatch='O', edgecolor='black', linewidth=1.2)
ax.hist(y_nondetec,bins=my_bins ,color='xkcd:rouge', alpha=0.6, hatch='.', edgecolor='black', linewidth=1.2)

ax.annotate(my_condition, xy=(0.015, 0.96), xycoords='axes fraction', # pts_concerned[to_plot]
size=15, ha='left', va='top',weight='bold',
bbox=dict(boxstyle='round',color='w',ec='k'))

ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5))


ax.set_xlabel(r'$ \Delta E_c$ [meV]')
ax.set_ylabel(r'Histogram')
# ax.set_ylim(365,545)
ax.grid()

tight_layout()

savefig(figname+'_cond'+str(my_condition)+'.png',dpi=600)

In [29]:
y

array([-22.55356777, -23.06663856, -19.09993267, -23.01902536,
       -16.41717955, -27.91413796, -24.44090508,  -9.07595517,
       -30.28335729, -26.19080316, -11.52014783, -13.72661721,
       -36.70497006, -19.20046538, -17.85505688, -15.74405256,
       -24.60518283,  -8.37319142, -12.41937931, -32.85891349,
       -29.91337287, -12.85233224, -18.35371161, -21.14506021,
       -19.03754399, -11.08734724, -36.47494191, -14.32207278,
       -11.41839006, -15.06560315, -20.551722  , -18.57154072,
       -18.27026513, -18.71142799, -27.53770675, -17.51469107,
       -17.06521836, -16.3298058 ,  -7.72380952, -21.11164664,
       -35.47740434, -21.8579618 , -27.63333997, -23.16851187,
       -19.40382131, -19.04988488,  -9.02098075, -21.95797639,
       -27.85515171, -28.23215053, -24.92966867, -30.02354583,
       -16.9136825 , -15.203937  , -18.79957084,  -9.92300481,
       -24.61602929, -13.27141596, -25.85712334, -16.67924466,
       -19.40765733, -15.82114006, -16.3503022 , -24.17

# Gamma Plasma

In [136]:
n0 = list(n0)
n0.index(min(n0))
n0.index(max(n0))

3

In [80]:
Len_moy*1e6

array([380.63441837, 388.87737695, 395.94693127, 402.34305919,
       273.30711815, 282.51192408, 289.41200279, 295.40532108,
       219.83442633, 227.99040842, 236.3928712 , 243.606327  ,
       188.17885618, 196.00437223, 202.69656474, 208.60346411,
       164.96736041, 173.38108147, 180.44780753, 185.93778415,
       132.61265899, 141.15457193, 148.65637591, 155.00307152,
       146.97393442, 155.66334381, 162.8463366 , 169.01810552])

In [135]:
Ray_moy*1e6

array([43.29845028, 38.2746571 , 34.75363717, 31.67696675, 52.06725592,
       46.26726688, 41.61739963, 37.86094572, 58.20645639, 51.68637849,
       46.52751801, 42.05260063, 63.53749071, 56.1400948 , 50.25857056,
       45.55413439, 68.08928757, 60.0854524 , 53.73456415, 48.45862255,
       76.13871346, 66.85036786, 59.62622519, 53.63422533, 72.18903036,
       63.65237501, 56.70773825, 51.12163728])

In [137]:
aws

array([8.86579330e-06, 8.22454308e-06, 7.75852535e-06, 7.33265488e-06,
       8.97759732e-06, 8.38999121e-06, 7.88113599e-06, 7.45018558e-06,
       8.99316765e-06, 8.40987632e-06, 7.93569066e-06, 7.49304099e-06,
       9.05267638e-06, 8.44962358e-06, 7.93698511e-06, 7.50514440e-06,
       9.07302031e-06, 8.48685549e-06, 7.98337209e-06, 7.52667693e-06,
       9.08863575e-06, 8.50878464e-06, 8.02144738e-06, 7.57951571e-06,
       9.07738858e-06, 8.50821668e-06, 7.99688584e-06, 7.55580824e-06])

In [143]:
Len_moy = mean(Len,axis=1)*1e-3
Ray_moy = mean(Ray,axis=1)*1e-3
N_ions = 1024
# gamma plasma
V = 4/3*pi*Ray_moy**2*Len_moy
n0 = N_ions/(V)
aws = ( 3/(4*pi*n0) )**(1/3)
temperature_ex = linspace(1e-4,1000,10000)

# determine the extremum density
indmin = list(n0).index(min(n0))
indmax = list(n0).index(max(n0))

gamma_plasma = []
for i,j in enumerate(aws):
    gamma_plasma.append(C_e**2/(4*pi*eps0*kb*temperature_ex*j))
    
# Plot Gamma plasma
figname = 'Gamma_plasma'
fig, ax = plt.subplots(1, 1,num=figname,clear=True,sharex=True,sharey=True)
fig.set_size_inches(8,6)

my_label = [n0[indmin],n0[indmax]]

for j,k in enumerate([indmin,indmax]):
    ax.plot(temperature_ex,gamma_plasma[k],label=f'{my_label[j]:.2e}')
plt.yscale("log")
plt.xscale("log")

ax.hlines(2,1e-5,1e4,linestyle=':',lw=2)
ax.hlines(278,1e-5,1e4,linestyle=':',lw=2)


locmaj = matplotlib.ticker.LogLocator(base=10,numticks=12) 
ax.xaxis.set_major_locator(locmaj)
ax.yaxis.set_major_locator(locmaj)
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
ax.xaxis.set_minor_locator(locmin)
ax.yaxis.set_minor_locator(locmin)

ax.grid(b=True, which='major', linestyle='-')
ax.grid(b=True, which='minor', linestyle='--')


ax.annotate('Crystal', xy=(0.97, 0.885), xycoords='axes fraction', # pts_concerned[to_plot]
size=15, ha='right', va='top',weight='bold',
bbox=dict(boxstyle='round',color='w',ec='k'))

ax.annotate('Liquid', xy=(0.97, 0.61), xycoords='axes fraction', # pts_concerned[to_plot]
size=15, ha='right', va='top',weight='bold',
bbox=dict(boxstyle='round',color='w',ec='k'))

ax.annotate('Gas', xy=(0.97, 0.235), xycoords='axes fraction', # pts_concerned[to_plot]
size=15, ha='right', va='top',weight='bold',
bbox=dict(boxstyle='round',color='w',ec='k'))

# ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
# ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=5))


ax.set_xlabel(r'Temperature [K]')
ax.set_ylabel(r'$\Gamma_p$')
ax.set_xlim(9e-5,1.2e3)
ax.legend(title='Density [#ions/m$^3$]',loc=3,fontsize=18)
# ax.grid()

tight_layout()

# savefig('gamma_p_vs_temperature',dpi=600)

In [144]:
# with different marker for each RF in the same curve

fname = 'detection_rate_vs_L_custommarkers'
cm = pylab.get_cmap('cividis')
fig = plt.figure(num=fname,clear=True)
fig.set_size_inches(11.69,8.27)

gs1 = gridspec.GridSpec(1,1)
gs1.update(left=0.11, right=0.88, top=0.98, bottom=0.17, wspace=0, hspace = 0)
ax1 = plt.subplot(gs1[0])

gs2 = gridspec.GridSpec(1, 1)
gs2.update(left=0.89, right=0.92, top=0.98 , bottom=0.17, wspace=0, hspace = 0)

# gs3 = gridspec.GridSpec(1, 1)
# gs3.update(left=0.89, right=0.92, top=0.98 , bottom=0.17, wspace=0, hspace = 0)


axb = plt.subplot(gs2[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
bounds = [0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(axb, cmap=cmap,
                                norm=norm,
                                ticks=np.arange(1,8), # np.arange(0.5,7.5)
                                spacing='proportional',
                                orientation='vertical')
cb2.set_label('$U_{DC}$ [V]')

fig = figure('detection_rate_vs_ion_density',clear=True)
ax1 = fig.add_subplot(111)
fig.set_size_inches(11.69, 6.1)

cm=pylab.get_cmap('cividis') 

# omegaz_2

for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    print(to_plot)
    ax1.errorbar(n0[min(to_plot):max(to_plot)+1],
                 detect_rate[min(to_plot):max(to_plot)+1],
             marker='D', ms=10, color=cm(k/6), markeredgecolor='k', linestyle=':',label=k+1)
ax1.set_xlabel('mean cloud density [#ions/m^3]')
ax1.set_ylabel('Detection rate [%]')
ax1.legend(title=r'$U_{DC}$',fontsize=15,loc=4)

for i, txt in enumerate(points_and_coord):
    if i in [16,24,20,12,15,11,13,25,21,27]:
        pass
    else:
        ax1.annotate(str(i), (n0[i]+1.5e12*(-1)**(i%5), detect_rate[i]*100+4.5*(-1)**(i%5)),
        color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
        ha='center',va='center',weight='bold',zorder = 1000)
    
ax1.annotate('16,24,\n20&12', (3.2e14,6.5),
color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
ha='center',va='center',weight='bold',zorder = 1000)

ax1.annotate('21&25', (3.9e14,-5),
color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
ha='center',va='center',weight='bold',zorder = 1000)

ax1.annotate('13', (4.02e14,5),
color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
ha='center',va='center',weight='bold',zorder = 1000)

ax1.annotate('15', (5.65e14,95),
color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
ha='center',va='center',weight='bold',zorder = 1000)

ax1.annotate('11', (5.7e14,104),
color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
ha='center',va='center',weight='bold',zorder = 1000)

ax1.annotate('27', (5.52e14,104.5),
color='xkcd:black',fontsize=10, # cm(1-(i//4)/6)
ha='center',va='center',weight='bold',zorder = 1000)

matplotlib.pyplot.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))

ax1.set_xlim(3.05e14,6.15e14)
ax1.set_ylim(-8.5,108)
plt.tight_layout()


# savefig('detection_rate_vs_ion_density',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

[0, 1, 2, 3]
[4, 5, 6, 7]
[8, 9, 10, 11]
[12, 13, 14, 15]
[16, 17, 18, 19]
[24, 25, 26, 27]
[20, 21, 22, 23]


H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:85: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 


In [146]:
N_ions = 1024
my_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,24,25,26,27,20,21,22,23]
omegax_2 = []
omegax_2bis = []
cloud_density_formula = zeros(28)
for k,l in enumerate(list(points_and_coord)):
    my_udc = Udc[int(l[2:4])]
    my_urf = Urf[int(l[7:])]
    print(l,my_order[k],my_udc,my_urf)
    q = 4 * C_e * my_urf / (m_Ca*r0**2*Omega**2)
    omegaz = sqrt( 2 * C_e * my_udc / (m_Ca*d0**2) )
    omegax_2.append(Omega**2 / 4 *  q**2/2)
    beta2 = q**2/(2-q**2) - 7*q**4/(32*4) + 29*q**6/(64*4*9)
    omegax_2bis.append(0.25*Omega**2 * a )
    omegar_2 = Omega**2 / 4 * ( q**2/2 - 0.5*(omegaz/Omega)**2 )
    cloud_density_formula[my_order[k]] = 2*m_Ca*eps0/C_e**2 * omegar_2

DC01_RF08 0 1 53.85
DC01_RF09 1 1 59.23
DC01_RF10 2 1 64.61
DC01_RF11 3 1 70.0
DC03_RF08 4 2 53.85
DC03_RF09 5 2 59.23
DC03_RF10 6 2 64.61
DC03_RF11 7 2 70.0
DC05_RF08 8 3 53.85
DC05_RF09 9 3 59.23
DC05_RF10 10 3 64.61
DC05_RF11 11 3 70.0
DC07_RF08 12 4 53.85
DC07_RF09 13 4 59.23
DC07_RF10 14 4 64.61
DC07_RF11 15 4 70.0
DC09_RF08 16 5 53.85
DC09_RF09 17 5 59.23
DC09_RF10 18 5 64.61
DC09_RF11 19 5 70.0
DC11_RF08 24 7 53.85
DC11_RF09 25 7 59.23
DC11_RF10 26 7 64.61
DC11_RF11 27 7 70.0
DC18_RF08 20 6 53.85
DC18_RF09 21 6 59.23
DC18_RF10 22 6 64.61
DC18_RF11 23 6 70.0


In [172]:
omegax[0:4] , 4*C_e/(m_Ca*r0**2*Omega**2)*array(Urf[8:12]) * Omega/(2*sqrt(2))

(array([2484067.11934794, 2775969.06119444, 3086217.18545825,
        3421863.47236089]),
 array([2334091.9098945 , 2567284.37925815, 2800476.84862179,
        3034102.76123704]))

In [189]:
# with different marker for each RF in the same curve

fname = 'detection_rate_vs_L_custommarkers_density'
cm = pylab.get_cmap('cividis')
fig = plt.figure(num=fname,clear=True)
fig.set_size_inches(11.69,8.27)

gs1 = gridspec.GridSpec(1,1)
gs1.update(left=0.11, right=0.72, top=0.98, bottom=0.17, wspace=0, hspace = 0)
ax1 = plt.subplot(gs1[0])

gs2 = gridspec.GridSpec(1, 1)
gs2.update(left=0.73, right=0.76, top=0.98 , bottom=0.17, wspace=0, hspace = 0)
gs3 = gridspec.GridSpec(1, 1)
gs3.update(left=0.84, right=0.841, top=0.98 , bottom=0.17, wspace=0, hspace = 0)

my_marker=['v','s','p','H']
my_label =[53.85, 59.23, 64.61, 70.00]
my_size =[225,150,225,225]

# for k in range(28//4):
#     to_plot = my_order[k*4:(1+k)*4]
#     print(to_plot)
#     ax1.errorbar(mean(cloud_density,axis=1)[min(to_plot):max(to_plot)+1],
#                  multiply(detect_rate,100)[min(to_plot):max(to_plot)+1],
#                  xerr=std_cloud_density[min(to_plot):max(to_plot)+1],
#              marker='D', ms=10, color=cm(k/6), markeredgecolor='k', linestyle=':',label=k+1)
# ax1.set_xlabel('mean cloud density [#ions/m^3]')
# ax1.set_ylabel('Detection rate [%]')
# ax1.legend(title=r'$U_{DC}$',fontsize=15,loc=4)

for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    
    x = n0[min(to_plot):max(to_plot)+1]
    y = array(detect_rate[min(to_plot):max(to_plot)+1])
    cluster = np.array([1,2,3,4])
    
    ax1.plot(x,y, marker='',color=cm(k/6),linestyle='--',lw=2)

    for kk in range(4):
        ax1.scatter(x[cluster==kk+1],y[cluster==kk+1],
                    marker=my_marker[kk],
                   s=my_size[kk], color=cm(k/6), edgecolor='k',zorder=1000)
#     ax1.errorbar(x,y,
#                  xerr=std_RoverL[min(to_plot):max(to_plot)+1],
#                  marker='', color=cm(k/6), linestyle='--')

debut = 0
fin   = 4
for kk in range(4):
    ax1.vlines(omegax[kk]**2*2*m_Ca*eps0/C_e**2,0,110,ls=':')
    
# for kk in Urf[8:12]:
#     print(kk)
#     uuu = 4*C_e/(m_Ca*r0**2*Omega**2)*array(Urf[8:12]) * Omega/(2*sqrt(2))
#     ax1.vlines(uuu**2*2*m_Ca*eps0/C_e**2,0,110,ls=':')
#     ax1.vlines(eps0/(m_Ca*Omega**2*r0**4)*kk**2,0,110,ls=':')

for kk in range(4):
    ax1.scatter(mean(omegax[kk::4])**2*2*m_Ca*eps0/C_e**2,115,
                    marker=my_marker[kk],
                   s=my_size[kk], color='k',zorder=1000)

omegax_2    

# ax1.vlines(omegax[1::4]**2*2*m_Ca*eps0/C_e**2,0,100)

for kk in range(4):
        ax1.scatter(0,0, marker=my_marker[kk],
                   s=225, color='k', edgecolor='k',label=my_label[kk],zorder=0)
handles, labels = ax1.get_legend_handles_labels()

# matplotlib.rcParams.update({'font.size': 20})

ax1.legend(handles[::-1], labels[::-1],
           title='$U_{RF}$', loc='lower right',fontsize=20)
    
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5e14))
    
ax1.set_xlabel(r'Cloud density [#ions/m$^3$]')
ax1.set_ylabel('Detection rate [%]')
ax1.set_xlim(2.5e14,6.49e14)

axb = plt.subplot(gs2[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
bounds = [0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(axb, cmap=cmap,
                                norm=norm,
                                ticks=np.arange(1,8), # np.arange(0.5,7.5)
                                spacing='proportional',
                                orientation='vertical')
cb2.set_label('$U_{DC}$ [V]')

axc = plt.subplot(gs3[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
cb3 = mpl.colorbar.ColorbarBase(axc, cmap=cmap,
                                norm=norm,
                                spacing='uniform',
                                orientation='vertical')
cb3.set_ticks([0,1,2,3,4,5,6,7])
cb3.set_ticklabels([ 90.8, 128, 157, 181,
       203, 240, 222])
cb3.set_label('$f_{z}$ [kHz]')

ax1.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 6.1)
matplotlib.rcParams.update({'font.size': 25})
savefig('Detec_dens_URF_UDC_fz',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

In [71]:
# with different marker for each RF in the same curve

fname = 'RF_vs_cloud_density'
cm = pylab.get_cmap('cividis')
fig = plt.figure(num=fname,clear=True)
fig.set_size_inches(11.69,8.27)

gs1 = gridspec.GridSpec(1,1)
gs1.update(left=0.11, right=0.72, top=0.98, bottom=0.17, wspace=0, hspace = 0)
ax1 = plt.subplot(gs1[0])

gs2 = gridspec.GridSpec(1, 1)
gs2.update(left=0.73, right=0.76, top=0.98 , bottom=0.17, wspace=0, hspace = 0)
gs3 = gridspec.GridSpec(1, 1)
gs3.update(left=0.84, right=0.87, top=0.98 , bottom=0.17, wspace=0, hspace = 0)

my_marker=['v','s','p','H']
my_label =[53.85, 59.23, 64.61, 70.00]
my_size  =[225,150,225,225]
my_urf   =[53.85,59.23,64.61,70]

# for k in range(28//4):
#     to_plot = my_order[k*4:(1+k)*4]
#     print(to_plot)
#     ax1.errorbar(mean(cloud_density,axis=1)[min(to_plot):max(to_plot)+1],
#                  multiply(detect_rate,100)[min(to_plot):max(to_plot)+1],
#                  xerr=std_cloud_density[min(to_plot):max(to_plot)+1],
#              marker='D', ms=10, color=cm(k/6), markeredgecolor='k', linestyle=':',label=k+1)
# ax1.set_xlabel('mean cloud density [#ions/m^3]')
# ax1.set_ylabel('Detection rate [%]')
# ax1.legend(title=r'$U_{DC}$',fontsize=15,loc=4)

for k in range(28//4):
    to_plot = my_order[k*4:(1+k)*4]
    
    x = array(my_urf)
    y = n0[min(to_plot):max(to_plot)+1]
    cluster = np.array([1,2,3,4])
    
    ax1.plot(x,y, marker='',color=cm(k/6),linestyle='--',lw=2)

    for kk in range(4):
        ax1.scatter(x[cluster==kk+1],y[cluster==kk+1],
                    marker=my_marker[kk],
                   s=my_size[kk], color=cm(k/6), edgecolor='k',zorder=1000)
#     ax1.errorbar(x,y,
#                  xerr=std_RoverL[min(to_plot):max(to_plot)+1],
#                  marker='', color=cm(k/6), linestyle='--')

for kk in range(4):
        ax1.scatter(0,0, marker=my_marker[kk],
                   s=225, color='k', edgecolor='k',label=my_label[kk],zorder=0)
handles, labels = ax1.get_legend_handles_labels()

# matplotlib.rcParams.update({'font.size': 20})

ax1.legend(handles[::-1], labels[::-1],
           title='$U_{RF}$', loc='lower right',fontsize=20)
    
# ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5e14))
    
ax1.set_xlabel(r'$U_{RF}$ [V]')
ax1.set_ylabel('Cloud density [#ions/m$^3$]')
ax1.set_xlim(50,75)
ax1.set_ylim(3e14,6.2e14)

axb = plt.subplot(gs2[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
bounds = [0.5,1.5,2.5,3.5,4.5,5.5,6.5,7.5]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
cb2 = mpl.colorbar.ColorbarBase(axb, cmap=cmap,
                                norm=norm,
                                ticks=np.arange(1,8), # np.arange(0.5,7.5)
                                spacing='proportional',
                                orientation='vertical')
cb2.set_label('$U_{DC}$ [V]')

axc = plt.subplot(gs3[0])
cmap = mpl.colors.ListedColormap([cm(k/6) for k in range(7)])
cb3 = mpl.colorbar.ColorbarBase(axc, cmap=cmap,
                                norm=norm,
                                spacing='uniform',
                                orientation='vertical')
cb3.set_ticks([0,1,2,3,4,5,6,7])
cb3.set_ticklabels([ 90.8, 128, 157, 181,
       203, 240, 222])
cb3.set_label('$f_{z}$ [kHz]')

ax1.grid(True, which="both")
# ax1.xaxis.set_major_locator(plt.MaxNLocator(25))
fig.set_size_inches(11.69, 6.1)
matplotlib.rcParams.update({'font.size': 25})
# savefig('Detec_dens_URF_UDC_fz',dpi=300)

# ax.scatter(x,y,
#         marker='D',s=400,edgecolor='black',zorder = 1000,
#         c = z, cmap=discrete_cmap(len(num_runs), 'rainbow'))

tight_layout()

H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:99: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
